In [1]:
import os, sys
ROOT_DIR = os.path.dirname(os.path.dirname(os.path.abspath('__file__')))
os.chdir(ROOT_DIR)
sys.path.append(ROOT_DIR)

In [2]:
from scripts import config as src
import pandas as pd
import numpy as np

import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import OneHotEncoder
import tensorflow as tf

# Import DeepExplain
from deepexplain.tensorflow import DeepExplain
import warnings
warnings.filterwarnings('ignore')

rand_state = 91

In [3]:
dataset              = 'reference_log1p.pck'
bio_knowledge        = 'pbk_circuit_hsa_sig.txt'
experiment           = 'exper_melanoma'
analysis             = 'activation'
split='train_test_split_tuningFalse_filtergeneFalse'
model_name ='circuits_signaling_2_1_layer'
model = tf.keras.models.load_model(filepath=f'./models/{experiment}/{split}/design_{model_name}_reference_log1p_Adam_relu_0.h5')
model.summary()


Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
layer1 (Dense)               (None, 1210)              21773950  
_________________________________________________________________
layer3 (Dense)               (None, 5)                 6055      
Total params: 21,780,005
Trainable params: 21,780,005
Non-trainable params: 0
_________________________________________________________________


In [4]:
df = pd.read_pickle(os.path.join('./data/processed/', experiment, dataset))
# df = df.sample(n=20, replace=True, random_state=1) # sampling
ohe = OneHotEncoder()
X_data = df.iloc[:, :-1].values
y_data = df.iloc[:, -1:].values
y_ohe = ohe.fit_transform(y_data).toarray()
# groups = y.reshape(1,-1)[0]
# y_cat = df['cell_type'].astype('category').cat.codes
print(X_data.shape)
print(y_data.shape)

print(df.groupby('cell_type').size())

(2761, 17994)
(2761, 1)
cell_type
B.cell         573
Macrophage     294
NK              64
T.CD4          599
T.CD8         1231
dtype: int64


In [5]:
df_pbk = pd.read_csv('./data/processed/pbk_circuit_hsa_sig.txt').set_index('symbol')

df_pbk_final = pd.merge(left=pd.DataFrame(df.columns[:-1]).set_index(0)
                        , right=df_pbk
                        , left_index=True
                        , right_index=True
                        , how='left').fillna(0)
# df_pbk_final = df_pbk_final.iloc[:, :10]
df_pbk_final

,P-hsa03320-62,P-hsa03320-45,P-hsa03320-43,P-hsa03320-23,P-hsa03320-32,P-hsa03320-8,P-hsa03320-7,P-hsa03320-9,P-hsa03320-39,P-hsa03320-38,...,P-hsa04976-38,P-hsa04976-37,P-hsa04976-36,P-hsa04976-34,P-hsa04976-41,P-hsa04976-16,P-hsa05100-55,P-hsa05100-33,P-hsa05100-15,P-hsa05100-24
0,,,,,,,,,,,,,,,,,,,,,
a1bg,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
a1cf,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
a2m,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
a2ml1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
a4galt,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
zyg11a,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
zyg11b,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
zyx,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [15]:
# predictions = model.predict(X_data)


df_circuits = []
circuit_gene_index = dict()
method_name = 'elrp'

with DeepExplain(session=tf.keras.backend.get_session() ) as de:
#     defining the input size which is the number of gene in given dataset, input_tensor--> X
    X = model.layers[0].input
#     the target layer size which shows the biological layer, target_tensor --> T
    T = model.layers[1].input
#     the mask for the filtering the target node in for analysis
    mask_ys = np.zeros((1, T.shape[1]))
    xs = X_data
    print(f'input_tensor, {X}\ntarget_tensor, {T}\nmask_ys, {mask_ys.shape}\nxs, {xs.shape}')
    
#     iterating the filtering mask for each node in the biological layer which defined as T
    for i_circuit in range(T.shape[1]):
        if i_circuit % 100 == 0:
            print(i_circuit)
#         filtering the target node in target layer with assigning value as 1
        mask_ys[:, i_circuit] = 1.0
#         compute attributions of each genes ('X') for each sample defined as 'xs' with defined mask 'T * mask_ys'
        explain = de.explain(method_name, T * mask_ys, X, xs)
#         getting the mean score of full dataset ( by given in xs) of each genes for filtered target node 
        df_circuits.append(np.mean(explain, axis=0))
    
#         additional information
#         the gene index information, it shows which genes are included for selected circuit
        gene_index = [i for i,x in enumerate(df_pbk_final.iloc[:, i_circuit].values) if x == 1]
        circuit_gene_index.update({i_circuit : gene_index})
    


input_tensor, Tensor("layer1_input:0", shape=(?, 17994), dtype=float32)
target_tensor, Tensor("layer1/Relu:0", shape=(?, 1210), dtype=float32)
mask_ys, (1, 1210)
xs, (2761, 17994)
0
100
200
300
400
500
600
700
800
900
1000
1100
1200


pd.DataFrame(df_circuits)

circuit_gene_index

In [23]:
pd.DataFrame(df_circuits).to_csv(f'./reports/deepexplain/{experiment}/{model_name}_{method_name}.csv')